In [7]:
import ConfigParser
import os
import pandas as pd

In [20]:
def get_dataframe(src_dir):
    
    # Get directories with valid/completed experiments
    test_dirs = sorted(os.listdir(src_dir))
    test_dir_dict = {}
    for ctr, curr in enumerate(test_dirs):
        if 'results.txt' in os.listdir(os.path.join(src_dir, curr)):
            test_dir_dict[ctr] = os.path.join(src_dir, curr)
     
    # Read cfg files in chosen dirs to create dict with summarized info
    summary_dicts = {}
    for curr in test_dir_dict:
        summary_dict = {}
        
        xx = ConfigParser.ConfigParser()
        cfg_files = [os.path.join(test_dir_dict[curr], x ) 
                     for x in os.listdir(test_dir_dict[curr]) if x[-4:] == '.cfg']
        xx.read(cfg_files)
        xx_dict = xx._sections
        summary_dict['Arch'] = xx_dict['NetParams']['arch_module']
        summary_dict['Optimizer'] = xx_dict['OptimizerParams']['optimizer']
        summary_dict['Data'] = xx_dict['ExptFiles']['data_generator']
        summary_dict['Epochs'] = xx_dict['ExptParams']['epochs']
        
        if 'MetricParams' in xx_dict:
            summary_dict['Acc_Metric']= xx_dict['MetricParams']['accuracy_metric']
            summary_dict['Encoding'] = xx_dict['EncodingModuleParams']['encoding_module'] + "_" + \
                                       xx_dict['Encoding']['nb_code_bits']
        else:
            enc_file = xx_dict['ExptFiles']['encoding_cfg']
            yy = ConfigParser.ConfigParser()
            yy.read(os.path.join('.',enc_file))
            yy_dict = yy._sections
            summary_dict['Encoding'] = yy_dict['EncodingModuleParams']['encoding_module'] + "_" + \
                                       yy_dict['Encoding']['nb_code_bits']
            summary_dict['Acc_Metric']= yy_dict['MetricParams']['accuracy_metric'] 

        with open(os.path.join(test_dir_dict[curr], 'results.txt'), 'r') as f:
            fl = f.readlines()
            tr_acc = []
            va_acc = []
            for curr_line in fl:
                info = curr_line.split(' ')
                tr_acc.append(float(info[3]))
                va_acc.append(float(info[9]))
        summary_dict['Top_Val_Acc'] = max(va_acc)
        summary_dict['Top_Train_Acc'] = max(tr_acc)
        summary_dicts[test_dir_dict[curr].split('/')[-1]] = summary_dict   
        
    # Create dataframe
    summary_df = pd.DataFrame.from_dict(summary_dicts).transpose()
    summary_df.style.set_properties(**{'text-align': 'center'})
    new_order = [7,6,4,0,5,3,1,2]
    summary_df = summary_df[summary_df.columns[new_order]]
    summary_df.sort_values(by=['Top_Val_Acc'], inplace=True, ascending=False)
    summary_df = summary_df[(summary_df.Top_Val_Acc > 0.20)]

    return summary_df    

In [21]:
get_dataframe('./tiny_imagenet_expts')

,Top_Val_Acc,Top_Train_Acc,Epochs,Acc_Metric,Optimizer,Encoding,Arch,Data
balin_tiny_imagenet_random_v207_*,0.4902,0.4677,20,ECOC_top_1,adam,rnd_encoding_200,tiny_imagenet_minigooglenet,tiny_imagenet_generator
balin_tiny_imagenet_random_v231_*,0.4792,0.4811,20,ECOC_top_1,adam,rnd_encoding_200,tiny_imagenet_minigooglenet,tiny_imagenet_generator
balin_tiny_imagenet_random_v182_*,0.4266,0.7388,50,hot_bit,adam,n_hot_encoding_200,milani_inception,tiny_imagenet_generator
balin_tiny_imagenet_random_v187_*,0.38,0.4699,20,hot_bit,adam,n_hot_encoding_200,milani_inception,tiny_imagenet_generator
balin_tiny_imagenet_random_v192_*,0.3639,0.4421,20,hot_bit,adam,n_hot_encoding_200,milani_inception,tiny_imagenet_generator
balin_tiny_imagenet_random_v232_*,0.3326,0.0236,20,ECOC_top_1,adam,rnd_encoding_200,tiny_imagenet_minigooglenet,tiny_imagenet_generator
balin_tiny_imagenet_random_v194_*,0.3205,0.3074,20,ECOC_top_1,adam,rnd_encoding_200,tiny_imagenet_minigooglenet,tiny_imagenet_generator
balin_tiny_imagenet_random_v183_*,0.2959,0.3301,500,ECOC_top_1,sgd,rnd_encoding_200,tiny_imagenet_minigooglenet,tiny_imagenet_generator
balin_tiny_imagenet_random_v190_*,0.2649,0.2763,50,ECOC_top_1,sgd,rnd_encoding_200,tiny_imagenet_minigooglenet,tiny_imagenet_generator
balin_tiny_imagenet_random_v255_*,0.2016,0.1937,10,ECOC_top_1,adam,rnd_encoding_200,tiny_imagenet_minigooglenet,tiny_imagenet_generator
